<a href="https://colab.research.google.com/github/Loyloyy/Hotel_Cancellation_Prediction/blob/master/Hotel_Analysis_(Tensorflow_Keras).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This kernel will showcase my work on using Tensorflow (Keras) to create a Deep Learning model to further improve the accuracy of the engineered dataset from the hotel analysis project.

I will be using both Tensorflow (Keras) and Tensorflow only in aother kernel. The reason for this is to allow myself to not only learn both but to also be able to create a model quickly using this Keras API and to learn how to create a custom in depth model only using Tensorflow.

I have decided to use Google colab as it allows me to free up my computer to work on other projects while letting this model train in the background without costing any speed.



In [0]:
import io
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
import functools
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix, auc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Firstly, importing the raw data

In [0]:
from google.colab import files
uploaded = files.upload()

Saving hotel_bookings.csv to hotel_bookings.csv


In [0]:
hotel_data = pd.read_csv(io.BytesIO(uploaded['hotel_bookings.csv']))

The next few steps involved what has already been concluded in the main hotel analysis project. I will then be preparing the data for the neural network right away.

In [0]:
final_data = hotel_data.copy()

In [0]:
le = preprocessing.LabelEncoder()

In [0]:
# Dropping the 'company' column
final_data = final_data.drop(['company'], axis = 1)

# Remove the null values in the country columns

indices = final_data.loc[pd.isna(final_data['country']), :].index  
final_data = final_data.drop(final_data.index[indices])  

final_data['country'] = le.fit_transform(final_data['country']) 

final_data = pd.get_dummies(data = final_data, columns = ['hotel','meal', 'market_segment'
                                                            , 'distribution_channel', 'reserved_room_type', 'deposit_type'
                                                            , 'assigned_room_type', 'customer_type', 'reservation_status'])

final_data = final_data.drop(columns = [ 'reservation_status_Canceled', 'reservation_status_Check-Out'
                                          , 'reservation_status_No-Show', 'arrival_date_day_of_month'
                                          , 'stays_in_weekend_nights', 'children', 'arrival_date_week_number'
                                          , 'arrival_date_week_number', 'arrival_date_month', 'agent'
                                          , 'reservation_status_date'], axis = 1)

label = final_data['is_canceled']


final_data = final_data.drop(['is_canceled'], axis =1)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(final_data, label, test_size = 0.20, random_state = 42)

As this kernel is mainly for testing and trying different approaches, I will be using an early stop function which will help to stop training the model when there is no longer an increase in accuracy.

The aim of this will be to prevent the neural network from overfitting.

In [0]:
earlystop_callback = EarlyStopping(monitor = 'accuracy', min_delta = 0.0001, patience=10)

Next will be setting up the neural network.

I have decided to go with 4 hidden layers as after testing with 1 to 10 layers of hidden layers, this number gave the highest average accuracy.

The main activation functions that were contended included relu, softplus and swish. However as relu gave the most constant accuracy, I decided to head with relu. I will be trying combination of different activation functions in a separate Tensorflow project.

The optimizer and loss functions were also tested together with the other publicly available ones provided in the Tensorflow library but the ones I used were the most consistent and resulted in the highest accuracy.

One interesting takeaway from this project was that to my knowledge, the last activation function is known to either include sigmoid for a binary output (2 options) and softmax for 3 or more output. However, in this situation, the softmax function consistently outperformed the sigmoid despite the output layer only having only 2 neurals (2 outputs).

In [0]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape = (63, )),    # input layer (1)
    keras.layers.Dense(250, activation = 'relu'),  # hidden layer (2)
    keras.layers.Dense(250, activation = 'relu'),  # hidden layer (3)
    keras.layers.Dense(250, activation = 'relu'),  # hidden layer (4)
    keras.layers.Dense(250, activation = 'relu'),  # hidden layer (5)
    keras.layers.Dense(2, activation = 'softmax') # output layer (6)
])

model.compile(optimizer = 'adam',
              loss = 'SparseCategoricalCrossentropy',
              metrics = ['accuracy'])

I decided to have an epoch of 1000 with early stopback to allow that function to better determine when to stop training as to prevent overfitting.

In [0]:
model.fit(x_train, y_train, epochs = 1000, learning rate, callbacks = [earlystop_callback]) 

Epoch 1/1000
2973/2973 [==============================] - 6s 2ms/step - loss: 0.7707 - accuracy: 0.7165
Epoch 2/1000
2973/2973 [==============================] - 6s 2ms/step - loss: 0.4589 - accuracy: 0.7799
Epoch 3/1000
2973/2973 [==============================] - 6s 2ms/step - loss: 0.4256 - accuracy: 0.7926
Epoch 4/1000
2973/2973 [==============================] - 6s 2ms/step - loss: 0.4090 - accuracy: 0.8002
Epoch 5/1000
2973/2973 [==============================] - 7s 2ms/step - loss: 0.3976 - accuracy: 0.8022
Epoch 6/1000
2973/2973 [==============================] - 6s 2ms/step - loss: 0.3864 - accuracy: 0.8071
Epoch 7/1000
2973/2973 [==============================] - 6s 2ms/step - loss: 0.3814 - accuracy: 0.8097
Epoch 8/1000
2973/2973 [==============================] - 6s 2ms/step - loss: 0.3794 - accuracy: 0.8120
Epoch 9/1000
2973/2973 [==============================] - 6s 2ms/step - loss: 0.3735 - accuracy: 0.8148
Epoch 10/1000
2973/2973 [==============================] - 6s 2m

From the training, we can already see that the neural network is performing much better compared to Sklearn's version in the original kernel

In [0]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose = 1) 

print('Test accuracy:', test_acc)

744/744 [==============================] - 1s 2ms/step - loss: 0.3446 - accuracy: 0.8377
Test accuracy: 0.8377276062965393


## Conclusion

Even with such a basic neural network, the performance has increase frastically as compared to the supervised models in the orginal kernel.

As such, I will be diving into the raw Tensorflow implementation in a separate kernel to hopefully increase the accuracy.

Thank you for your time and if you have any feedbacks, feel free to reach out to me too!